In [41]:
import os
import numpy as np
import pandas as pd

from bayes_opt import BayesianOptimization
import xgboost as xgb

# Scikit-learn
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier

pd.options.display.max_columns = 999
np.random.seed(123)
random_state = 2017
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

In [2]:
def load_data(ravel=True):
    x_train = np.loadtxt("x_train.csv", delimiter = ",", skiprows = 1)
    x_test  = np.loadtxt("x_test.csv", delimiter = ",", skiprows = 1)    
    y_train = np.loadtxt("y_train.csv", delimiter = ",", skiprows = 1)
    
    
    # remove the first column(Id)
    x_train = x_train[:,1:]
    x_test  = x_test[:,1:]
    y_train = y_train[:,1:]

    num_genes_train = x_train.shape[0] / 100
    num_genes_test  = x_test.shape[0] / 100

    print("Train / test data has %d / %d genes." % \
          (num_genes_train, num_genes_test))

    x_train = np.split(x_train, num_genes_train)
    x_test  = np.split(x_test, num_genes_test)

    if ravel:
        # Reshape by raveling each 100x5 array into a 500-length vector
        x_train = [g.ravel() for g in x_train]
        x_test  = [g.ravel() for g in x_test]
    
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test  = np.array(x_test)
    y_train = np.ravel(y_train)
    
    
    print("x_train shape is %s" % str(x_train.shape))    
    print("y_train shape is %s" % str(y_train.shape))
    print("x_test shape is %s" % str(x_test.shape))
    
    return(x_train, y_train, x_test)

def minmax_scale(array, minimum=0, maximum=1):
    array_std = (array - array.min(axis=0)) / (array.max(axis=0) - array.min(axis=0))
    return array_std * (maximum - minimum) + minimum



In [3]:
x_train, y_train, x_test = load_data()
x_train_df = pd.DataFrame(x_train)
x_test_df = pd.DataFrame(x_test)

Loading data...
All files loaded. Preprocessing...
Train / test data has 15485 / 3871 genes.
x_train shape is (15485, 500)
y_train shape is (15485,)
x_test shape is (3871, 500)
Data preprocessing done...


In [4]:
x_train_100_5, y_train, x_test_100_5 = load_data(ravel=False)
x_train_100_5 = minmax_scale(x_train_100_5)
x_test_100_5 = minmax_scale(x_test_100_5)

Loading data...
All files loaded. Preprocessing...
Train / test data has 15485 / 3871 genes.
x_train shape is (15485, 100, 5)
y_train shape is (15485,)
x_test shape is (3871, 100, 5)
Data preprocessing done...


In [5]:
def get_stats(row, index=0, stat='mean'):
    if stat == 'mean':
        return row[index:500:5].mean()
    elif stat == 'perc0': 
        return np.mean(row[index:500:5] == 0)
    elif stat == 'std':
        return np.std(row[index:500:5])
    else:
        raise ValueError(stat + ' currently not supported.')

In [6]:
markers = ['H3K4me3', 'H3K4me1', 'H3K36me3', 'H3K9me3', 'H3K27me3']
dataframes = [x_train_df, x_test_df]
stats = ['mean', 'perc0', 'std']

for df in dataframes:
    for stat in stats:
        for i, j in enumerate(markers):
            df[j + '_' + stat] = df.apply(get_stats, args=(i, stat), axis=1)

In [7]:
xg_train = xgb.DMatrix(x_train_df, label=y_train)

In [9]:
def xgb_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample,
                 gamma,
                 alpha):

    params['min_child_weight'] = int(min_child_weight)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    params['alpha'] = max(alpha, 0)

    cv_result = xgb.cv(params, xg_train, num_boost_round=num_rounds, 
                       nfold=5, seed=random_state, stratified=True, 
                       metrics='auc', callbacks=[xgb.callback.early_stop(100)])

    return cv_result['test-auc-mean'].values[-1]


num_rounds = 3000
num_iter = 2
init_points = 5
params = {
        'objective': 'binary:logistic',
        'eta': .01,
        'silent': 1,
        'verbose_eval': True,
        'seed': random_state
    }

xgbBO = BayesianOptimization(xgb_evaluate, {'min_child_weight': (1, 20),
                                            'colsample_bytree': (.1, 1),
                                            'max_depth': (1, 40),
                                            'subsample': (.6, 1),
                                            'gamma': (0, 8),
                                            'alpha': (0, 8),
                                            })

xgbBO.maximize(init_points=init_points, n_iter=num_iter)

Initialization
---------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |     alpha |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   subsample | 
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
Stopping. Best iteration:
[1844]	train-auc:0.951736+0.00100339	test-auc:0.918979+0.00698068

    1 | 03m01s |    0.91898 |    4.8936 |             0.5854 |    0.3563 |      3.4288 |             4.5143 |      0.6743 | 
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
Stopping. Best iteration:
[876]	train-auc:0.999539+3.55562e-05	test-auc:0.919992+0.00618546

    2 | 04m19s |    0.91999 |    2.4851 |             0.4040 |    0.4793 |     17.1299 |             2.0074 |      0.8557 | 
Multiple eval me

C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


Bayesian Optimization
---------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |     alpha |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   subsample | 
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
Stopping. Best iteration:
[1304]	train-auc:0.998937+4.51442e-05	test-auc:0.917774+0.00703591

    6 | 13m29s |    0.91777 |    7.9754 |             0.8993 |    0.0578 |     39.3562 |             3.6012 |      0.9260 | 


C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([  1.85085721e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
    7 | 07m46s |    0.91884 |    7.9573 |             0.1652 |    7.9872 |     19.5067 |             2.1825 |      0.9468 | 


In [10]:
bayes_params3 = {
    'objective': 'binary:logistic',
    'eta': .01,
    'alpha': 0.9939,
    'colsample_bytree': .1066,
    'gamma': 1.8422,
    'max_depth': 39,
    'min_child_weight': 7,
    'subsample': .8101,
    'seed': random_state,
    'silent': 1
}

In [12]:
xgb_cv = xgb.cv(bayes_params3, xg_train, num_boost_round=10000, early_stopping_rounds=200, nfold=5,
                stratified=True, verbose_eval=True, metrics='auc', seed=random_state)

[0]	train-auc:0.906633+0.00581156	test-auc:0.886849+0.0117642
[1]	train-auc:0.920163+0.00250473	test-auc:0.898137+0.00515209
[2]	train-auc:0.925403+0.00227319	test-auc:0.90277+0.00477192
[3]	train-auc:0.927815+0.00202068	test-auc:0.906217+0.00400852
[4]	train-auc:0.929821+0.0019787	test-auc:0.907839+0.00465316
[5]	train-auc:0.931135+0.0023938	test-auc:0.909536+0.00521998
[6]	train-auc:0.932222+0.00259713	test-auc:0.909966+0.00494245
[7]	train-auc:0.93317+0.00254182	test-auc:0.9106+0.00500454
[8]	train-auc:0.933745+0.00239626	test-auc:0.911318+0.00439832
[9]	train-auc:0.934476+0.00196349	test-auc:0.91147+0.0043765
[10]	train-auc:0.934874+0.00171577	test-auc:0.911797+0.0039877
[11]	train-auc:0.935287+0.00167982	test-auc:0.912434+0.00408773
[12]	train-auc:0.93578+0.00161783	test-auc:0.912655+0.00418142
[13]	train-auc:0.936024+0.0015447	test-auc:0.9128+0.00388814
[14]	train-auc:0.936242+0.00151727	test-auc:0.912872+0.00381564
[15]	train-auc:0.936635+0.00157077	test-auc:0.913161+0.00409302


[129]	train-auc:0.950795+0.000854917	test-auc:0.917896+0.0054143
[130]	train-auc:0.950892+0.000854655	test-auc:0.917912+0.00541602
[131]	train-auc:0.951003+0.000850877	test-auc:0.917932+0.00542791
[132]	train-auc:0.951093+0.000854805	test-auc:0.917959+0.00544311
[133]	train-auc:0.951193+0.000860324	test-auc:0.917971+0.00546522
[134]	train-auc:0.951298+0.000844889	test-auc:0.917974+0.00548146
[135]	train-auc:0.951406+0.00085463	test-auc:0.918023+0.00550234
[136]	train-auc:0.951514+0.000855068	test-auc:0.918016+0.00548969
[137]	train-auc:0.951612+0.000862156	test-auc:0.918022+0.00553448
[138]	train-auc:0.951727+0.000869399	test-auc:0.918018+0.005549
[139]	train-auc:0.951835+0.000861503	test-auc:0.918065+0.00552501
[140]	train-auc:0.951942+0.000868815	test-auc:0.918071+0.00551986
[141]	train-auc:0.952044+0.000851723	test-auc:0.918057+0.00549599
[142]	train-auc:0.952141+0.00085892	test-auc:0.918026+0.00551311
[143]	train-auc:0.952245+0.000841752	test-auc:0.918049+0.00551631
[144]	train-auc

[254]	train-auc:0.962379+0.000560677	test-auc:0.919401+0.00556288
[255]	train-auc:0.962465+0.000563277	test-auc:0.91943+0.00554441
[256]	train-auc:0.962545+0.000567815	test-auc:0.919429+0.00557524
[257]	train-auc:0.962633+0.000555065	test-auc:0.91942+0.00558308
[258]	train-auc:0.962706+0.000552212	test-auc:0.91943+0.00556146
[259]	train-auc:0.962786+0.000554656	test-auc:0.919439+0.00555019
[260]	train-auc:0.962873+0.00054962	test-auc:0.919451+0.00554684
[261]	train-auc:0.962957+0.000546157	test-auc:0.919456+0.00556039
[262]	train-auc:0.963034+0.000546678	test-auc:0.919445+0.00555902
[263]	train-auc:0.963117+0.000550226	test-auc:0.919435+0.00558106
[264]	train-auc:0.963195+0.000551163	test-auc:0.919449+0.0055745
[265]	train-auc:0.963282+0.000546471	test-auc:0.919461+0.00558696
[266]	train-auc:0.963365+0.000552255	test-auc:0.919492+0.00560126
[267]	train-auc:0.963445+0.000549405	test-auc:0.919499+0.00558808
[268]	train-auc:0.963527+0.000548407	test-auc:0.919509+0.00559381
[269]	train-auc

[380]	train-auc:0.971882+0.000446778	test-auc:0.92025+0.00555242
[381]	train-auc:0.971945+0.0004468	test-auc:0.920247+0.0055491
[382]	train-auc:0.972012+0.000439512	test-auc:0.920264+0.00555603
[383]	train-auc:0.972078+0.000439117	test-auc:0.92026+0.00553883
[384]	train-auc:0.97214+0.000434953	test-auc:0.920255+0.00553669
[385]	train-auc:0.972206+0.000424675	test-auc:0.920271+0.00552971
[386]	train-auc:0.972276+0.000424547	test-auc:0.920283+0.00554048
[387]	train-auc:0.972344+0.000421929	test-auc:0.920285+0.00554501
[388]	train-auc:0.972407+0.000425314	test-auc:0.92029+0.00554711
[389]	train-auc:0.972471+0.000425887	test-auc:0.920277+0.00555621
[390]	train-auc:0.972532+0.000418545	test-auc:0.920271+0.00554091
[391]	train-auc:0.972593+0.000416937	test-auc:0.920286+0.00554092
[392]	train-auc:0.972656+0.000415232	test-auc:0.920283+0.00553862
[393]	train-auc:0.972723+0.000410344	test-auc:0.920309+0.00552557
[394]	train-auc:0.972793+0.000411907	test-auc:0.920307+0.00554183
[395]	train-auc:0

[505]	train-auc:0.979433+0.000322864	test-auc:0.920762+0.00568679
[506]	train-auc:0.979485+0.000322234	test-auc:0.92077+0.00568264
[507]	train-auc:0.979538+0.000323544	test-auc:0.920773+0.00567486
[508]	train-auc:0.97959+0.000324053	test-auc:0.920777+0.0056744
[509]	train-auc:0.979641+0.000324113	test-auc:0.920789+0.00567464
[510]	train-auc:0.979693+0.00032308	test-auc:0.920807+0.00566845
[511]	train-auc:0.979745+0.00032313	test-auc:0.920792+0.00567144
[512]	train-auc:0.979795+0.000324268	test-auc:0.920797+0.00567263
[513]	train-auc:0.979848+0.000324713	test-auc:0.920806+0.00566789
[514]	train-auc:0.979899+0.000323063	test-auc:0.920806+0.0056775
[515]	train-auc:0.97995+0.00032073	test-auc:0.920818+0.00567062
[516]	train-auc:0.980005+0.000317473	test-auc:0.920815+0.00566107
[517]	train-auc:0.980058+0.00031644	test-auc:0.920807+0.00565853
[518]	train-auc:0.980105+0.000311976	test-auc:0.920821+0.0056609
[519]	train-auc:0.980153+0.000308407	test-auc:0.92082+0.00566877
[520]	train-auc:0.980

[630]	train-auc:0.985122+0.000184511	test-auc:0.920981+0.00567349
[631]	train-auc:0.985164+0.000185444	test-auc:0.92099+0.00567721
[632]	train-auc:0.985205+0.000187782	test-auc:0.92098+0.00567609
[633]	train-auc:0.985241+0.000190132	test-auc:0.920975+0.00567549
[634]	train-auc:0.985281+0.000190161	test-auc:0.920975+0.00566819
[635]	train-auc:0.985319+0.000188284	test-auc:0.920973+0.00567292
[636]	train-auc:0.985362+0.000186057	test-auc:0.920976+0.00568364
[637]	train-auc:0.985396+0.000187169	test-auc:0.920985+0.00568582
[638]	train-auc:0.985434+0.000186356	test-auc:0.920994+0.00567831
[639]	train-auc:0.985473+0.000184563	test-auc:0.920995+0.00568506
[640]	train-auc:0.985511+0.000179461	test-auc:0.921005+0.00569306
[641]	train-auc:0.985548+0.000180485	test-auc:0.921002+0.00568052
[642]	train-auc:0.985587+0.000177177	test-auc:0.921008+0.00568552
[643]	train-auc:0.985624+0.000179635	test-auc:0.921002+0.00567158
[644]	train-auc:0.98566+0.00017957	test-auc:0.921001+0.00567299
[645]	train-au

[755]	train-auc:0.989297+0.000131175	test-auc:0.921016+0.00574288
[756]	train-auc:0.989327+0.000131203	test-auc:0.921001+0.00574251
[757]	train-auc:0.989353+0.000132587	test-auc:0.921009+0.00573092
[758]	train-auc:0.989385+0.000130205	test-auc:0.921009+0.00573397
[759]	train-auc:0.989413+0.000130303	test-auc:0.921006+0.00572679
[760]	train-auc:0.989442+0.00012933	test-auc:0.921003+0.00573041
[761]	train-auc:0.989471+0.000129423	test-auc:0.920994+0.00574082
[762]	train-auc:0.9895+0.000129396	test-auc:0.921004+0.00573168
[763]	train-auc:0.989528+0.000128752	test-auc:0.921012+0.00573684
[764]	train-auc:0.989555+0.00012999	test-auc:0.921014+0.00573126
[765]	train-auc:0.989582+0.000130228	test-auc:0.921026+0.00571745
[766]	train-auc:0.989611+0.000128857	test-auc:0.921016+0.00572023
[767]	train-auc:0.989642+0.000130843	test-auc:0.921013+0.00572565
[768]	train-auc:0.989671+0.000129764	test-auc:0.921013+0.00571843
[769]	train-auc:0.989699+0.000129552	test-auc:0.921009+0.00570233
[770]	train-au

[881]	train-auc:0.992297+7.32759e-05	test-auc:0.921073+0.0057871
[882]	train-auc:0.992318+7.12388e-05	test-auc:0.921072+0.00577681
[883]	train-auc:0.992337+7.05833e-05	test-auc:0.921087+0.00578779
[884]	train-auc:0.992358+6.79618e-05	test-auc:0.921088+0.00578374
[885]	train-auc:0.992378+6.998e-05	test-auc:0.921089+0.00578761
[886]	train-auc:0.992398+7.11129e-05	test-auc:0.921093+0.0057794
[887]	train-auc:0.99242+6.90478e-05	test-auc:0.921099+0.00579002
[888]	train-auc:0.992439+6.979e-05	test-auc:0.921101+0.0057989
[889]	train-auc:0.992458+7.1051e-05	test-auc:0.921101+0.00579993
[890]	train-auc:0.992479+7.2522e-05	test-auc:0.921104+0.00580541
[891]	train-auc:0.992497+7.07152e-05	test-auc:0.921104+0.0057972
[892]	train-auc:0.992517+7.22617e-05	test-auc:0.921105+0.00580435
[893]	train-auc:0.992538+7.01524e-05	test-auc:0.921109+0.00580468
[894]	train-auc:0.992556+6.9681e-05	test-auc:0.921106+0.00580556
[895]	train-auc:0.992576+7.18704e-05	test-auc:0.921103+0.00581199
[896]	train-auc:0.9925

[1006]	train-auc:0.994401+4.18426e-05	test-auc:0.921089+0.00591703
[1007]	train-auc:0.994413+4.15375e-05	test-auc:0.92109+0.00591265
[1008]	train-auc:0.994427+4.08039e-05	test-auc:0.9211+0.00591129
[1009]	train-auc:0.994441+4.09458e-05	test-auc:0.921088+0.00591403
[1010]	train-auc:0.994455+3.98196e-05	test-auc:0.921096+0.00590471
[1011]	train-auc:0.994467+4.08245e-05	test-auc:0.921083+0.0059017
[1012]	train-auc:0.99448+4.13057e-05	test-auc:0.921088+0.00591626
[1013]	train-auc:0.994494+4.0802e-05	test-auc:0.921098+0.00591417
[1014]	train-auc:0.994507+4.01766e-05	test-auc:0.92109+0.00592358
[1015]	train-auc:0.99452+3.82905e-05	test-auc:0.921103+0.00592176
[1016]	train-auc:0.994534+3.79051e-05	test-auc:0.921098+0.0059205
[1017]	train-auc:0.994547+3.76117e-05	test-auc:0.921096+0.00593133
[1018]	train-auc:0.994561+3.79231e-05	test-auc:0.921094+0.00592795
[1019]	train-auc:0.994574+3.70546e-05	test-auc:0.921096+0.00592162
[1020]	train-auc:0.994587+3.77921e-05	test-auc:0.921099+0.00592978
[102

In [42]:
x_train_df_meta = x_train_df.copy()
x_train_df_meta['XGB'] = np.nan
x_train_df_meta['LR'] = np.nan
x_train_df_meta['ET'] = np.nan
x_train_df_meta['RF'] = np.nan

x_test_df_meta = x_test_df.copy()
x_test_df_meta['XGB'] = np.nan
x_test_df_meta['LR'] = np.nan
x_test_df_meta['ET'] = np.nan
x_test_df_meta['RF'] = np.nan

In [43]:
clfs = [LogisticRegression(C=.01, penalty='l1'),
       ExtraTreesClassifier(n_estimators=800, n_jobs=2),
       RandomForestClassifier(n_estimators=800, n_jobs=2, criterion='entropy')]

clf_names = ['LR', 'ET', 'RF']

In [44]:
num_fold = 0
n_epochs = 2

CNN_test_preds = []
RNN_test_preds = []

neural_nets = ['CNN', 'RNN']

for train_index, test_index in cv.split(x_train_df, y_train):
    
    # Valitaan data sekä neuroverkoille että muille käytettäville malleille
    X_cvtrain = x_train_df.iloc[train_index]
    X_cvtest = x_train_df.iloc[test_index]
    X_cvtrain_100_5 = x_train_100_5[train_index]
    X_cvtest_100_5 = x_train_100_5[test_index]
    y_cvtrain = y_train[train_index]
    y_cvtest = y_train[test_index]
    
    num_fold += 1
    
    # Valmiiksi oltiin luotu viisi kansiota hakemistoon, yksi kutakin kierrosta varten
    directory = ('/Users/shrey/Desktop/summer/gene prediction/Kaggle-gene-expression-master/Fold'+ str(num_fold)) 
    if not os.path.exists(directory):
        os.makedirs(directory)
    os.chdir(directory)
    
    # Koulutetaan molemmat neuroverkot loopissa, CNN ja CNN+RNN
    
    # Rullataan scikit-learn mallit läpi ja ennustetaan jokaisella
    for i, clf in enumerate(clfs):
        clf.fit(X_cvtrain, y_cvtrain)
        clf_pred = clf.predict_proba(X_cvtest)[:, 1]
        x_train_df_meta.loc[test_index, clf_names[i]] = clf_pred
        
        print('Fold', num_fold, clf_names[i], 'CV AUC:', roc_auc_score(y_cvtest, clf_pred))
    
    # Koulutetaan XGBoost-malli aiemmin löydetyillä parametreilla
    xg_cvtrain = xgb.DMatrix(X_cvtrain, label=y_cvtrain)
    xg_cvtest = xgb.DMatrix(X_cvtest)
    
    xgb_model = xgb.train(bayes_params3, xg_cvtrain, num_boost_round=1184)
    xgb_cvpred = xgb_model.predict(xg_cvtest)
    x_train_df_meta.loc[test_index, 'XGB'] = xgb_cvpred
    print('Fold', num_fold, 'XGB CV AUC:', roc_auc_score(y_cvtest, xgb_cvpred))
    
    print('Fold', num_fold, 'completed.')
    print(50 * '-')

print('Train meta filled. Thank you.')
os.chdir('/Users/shrey/Desktop/summer/gene prediction/Kaggle-gene-expression-master')

Fold 1 LR CV AUC: 0.917492639802
Fold 1 ET CV AUC: 0.921853119457
Fold 1 RF CV AUC: 0.925319414252
Fold 1 XGB CV AUC: 0.927216561529
Fold 1 completed.
--------------------------------------------------
Fold 2 LR CV AUC: 0.914304957256
Fold 2 ET CV AUC: 0.918060491991
Fold 2 RF CV AUC: 0.921160860751
Fold 2 XGB CV AUC: 0.923885926101
Fold 2 completed.
--------------------------------------------------
Fold 3 LR CV AUC: 0.909510452298
Fold 3 ET CV AUC: 0.917681535845
Fold 3 RF CV AUC: 0.920647531159
Fold 3 XGB CV AUC: 0.924156096764
Fold 3 completed.
--------------------------------------------------
Fold 4 LR CV AUC: 0.913788577784
Fold 4 ET CV AUC: 0.916791546951
Fold 4 RF CV AUC: 0.918991609965
Fold 4 XGB CV AUC: 0.92067212092
Fold 4 completed.
--------------------------------------------------
Fold 5 LR CV AUC: 0.895832107483
Fold 5 ET CV AUC: 0.904875860443
Fold 5 RF CV AUC: 0.909656991314
Fold 5 XGB CV AUC: 0.910069085818
Fold 5 completed.
------------------------------------------

In [45]:
epoch_auc

0.90648814462215588

In [46]:
x_train_df_meta.isnull().sum()

0                 0
1                 0
2                 0
3                 0
4                 0
5                 0
6                 0
7                 0
8                 0
9                 0
10                0
11                0
12                0
13                0
14                0
15                0
16                0
17                0
18                0
19                0
20                0
21                0
22                0
23                0
24                0
25                0
26                0
27                0
28                0
29                0
                 ..
489               0
490               0
491               0
492               0
493               0
494               0
495               0
496               0
497               0
498               0
499               0
H3K4me3_mean      0
H3K4me1_mean      0
H3K36me3_mean     0
H3K9me3_mean      0
H3K27me3_mean     0
H3K4me3_perc0     0
H3K4me1_perc0     0
H3K36me3_perc0    0


In [47]:
x_train_df_meta.to_csv('train_meta.csv', index=False)

In [48]:
LR = LogisticRegression()
used_cols = ['XGB', 'LR', 'ET', 'RF']

cross_val_score(LR, x_train_df_meta[used_cols], y_train, scoring='roc_auc', cv=cv).mean()

0.92099685379871321

In [49]:
stat_cols = x_train_df_meta.columns[500:-7].values
used_cols = ['XGB','LR', 'ET','H3K27me3_std', 
             'H3K27me3_mean', 'H3K4me1_perc0', 'H3K9me3_std']

print('Baseline:', cross_val_score(LR, x_train_df_meta[used_cols], 
                      y_train, scoring='roc_auc', cv=cv).mean())

# Printataan uudet CV-pisteet featureita yksi kerrallaan lisäillen
for i in range(len(stat_cols)):
    print(stat_cols[i], cross_val_score(LR, x_train_df_meta[used_cols + [stat_cols[i]]],
                                        y_train, scoring='roc_auc', cv=cv).mean())

Baseline: 0.920723712877
H3K4me3_mean 0.919912882641
H3K4me1_mean 0.920257822472
H3K36me3_mean 0.92059741298
H3K9me3_mean 0.920386984768
H3K27me3_mean 0.920729969464
H3K4me3_perc0 0.919964759025
H3K4me1_perc0 0.920730470505
H3K36me3_perc0 0.92045173902
H3K9me3_perc0 0.920225272155
H3K27me3_perc0 0.920466890476
H3K4me3_std 0.91989614191
H3K4me1_std 0.920249123134


In [50]:
xgbs = []
xg_train = xgb.DMatrix(x_train_df, label=y_train)
xg_test = xgb.DMatrix(x_test_df)

for i in range(1, 8):
    complete_xgb_model = xgb.train(bayes_params3, xg_train, num_boost_round=int(1184/.8))
    xgb_pred = complete_xgb_model.predict(xg_test)
    xgbs.append(xgb_pred)
    
x_test_df_meta['XGB'] = np.mean(xgbs, axis=0)

In [51]:
final_clfs = [LogisticRegression(C=.01, penalty='l1'),
              ExtraTreesClassifier(n_estimators=800, n_jobs=2)]
final_clf_names = ['LR', 'ET']

x_test_df_meta.drop('RF', axis=1, inplace=True) # Random forest huomattiin haitalliseksi ensemblessä


for i, clf in enumerate(final_clfs):
    clf.fit(x_train_df, y_train)
    x_test_df_meta[final_clf_names[i]] = clf.predict_proba(x_test_df)[:, 1]

In [52]:
xg_train_meta = xgb.DMatrix(x_train_df_meta[used_cols], label=y_train)
xg_test_meta = xgb.DMatrix(x_test_df_meta[used_cols])

In [53]:
def xgb_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample,
                 gamma,
                 alpha):

    params['min_child_weight'] = int(min_child_weight)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    params['alpha'] = max(alpha, 0)

    cv_result = xgb.cv(params, xg_train_meta, num_boost_round=num_rounds, 
                       nfold=5, seed=random_state, stratified=True, 
                       metrics='auc', callbacks=[xgb.callback.early_stop(100)])

    return cv_result['test-auc-mean'].values[-1]


num_rounds = 3000
num_iter = 120
init_points = 5
params = {
        'objective': 'binary:logistic',
        'eta': .01,
        'silent': 1,
        'verbose_eval': True,
        'seed': random_state
    }

xgbBO = BayesianOptimization(xgb_evaluate, {'min_child_weight': (1, 20),
                                            'colsample_bytree': (.1, 1),
                                            'max_depth': (1, 40),
                                            'subsample': (.6, 1),
                                            'gamma': (0, 8),
                                            'alpha': (0, 8),
                                            })

xgbBO.maximize(init_points=init_points, n_iter=num_iter)

Initialization
---------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |     alpha |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   subsample | 
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
Stopping. Best iteration:
[1241]	train-auc:0.927614+0.0014028	test-auc:0.919835+0.00600617

    1 | 00m29s |    0.91983 |    5.7595 |             0.3749 |    2.5662 |      5.6199 |             3.1571 |      0.6286 | 
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
Stopping. Best iteration:
[296]	train-auc:0.924138+0.00143912	test-auc:0.920761+0.00611824

    2 | 00m12s |    0.92076 |    6.1588 |             0.8934 |    4.9824 |     20.2022 |             4.8368 |      0.6409 | 
Multiple eval metr

C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([  8.15276775e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 46, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


Bayesian Optimization
---------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |     alpha |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   subsample | 
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
Stopping. Best iteration:
[115]	train-auc:0.946955+0.00176823	test-auc:0.917526+0.00572783

    6 | 00m20s |    0.91753 |    0.4389 |             0.3289 |    0.1813 |     39.7229 |             1.2710 |      0.6329 | 
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
Stopping. Best iteration:
[575]	train-auc:0.922843+0.00150042	test-auc:0.920631+0.00622723

    7 | 00m24s |    0.92063 |    7.5140 |             0.9312 |    7.9824 |      2.0596 |            14.6366 |      0.9892 | 
Multiple ev

C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -8.66224818e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
Stopping. Best iteration:
[402]	train-auc:0.923552+0.00144994	test-auc:0.92062+0.00621686

   10 | 00m36s |    0.92062 |    7.5552 |             0.9311 |    7.6344 |     17.3634 |             1.4539 |      0.9942 | 


...

C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00222728]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 46, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.20877816]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.00038701]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 59, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.

Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
Stopping. Best iteration:
[275]	train-auc:0.923249+0.00141839	test-auc:0.920832+0.00611038

  124 | 01m48s |    0.92083 |    5.5531 |             0.9610 |    7.0624 |     30.8178 |             8.3332 |      0.7133 | 


C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00942021]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 66, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.00133143]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 70, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00287347]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 75, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.

Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
Stopping. Best iteration:
[430]	train-auc:0.923284+0.00148099	test-auc:0.920744+0.00627542

  125 | 01m38s |    0.92074 |    6.1349 |             0.9474 |    7.2599 |     10.6732 |            17.2274 |      0.6627 | 


C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.010752]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -2.57194042e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 71, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -7.87079334e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 57, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


In [54]:
bayes_params_final = {
    'objective': 'binary:logistic',
    'eta': .01,
    'alpha': .0748,
    'colsample_bytree': .8819,
    'gamma': .8596,
    'max_depth': 1,
    'min_child_weight': 2,
    'subsample': .6542,
    'seed': random_state,
    'silent': 1
}

In [57]:
final_xgb_ = xgb.train(bayes_params_final, xg_train_meta, num_boost_round=int(912/.8))
xgb_pred = final_xgb_.predict(xg_test_meta)

In [58]:
pred_df = pd.DataFrame({'GeneId': np.arange(1, x_test.shape[0] + 1), 
                        'Prediction': xgb_pred})
pred_df.to_csv('submission.csv', index=False)